In [1]:
import sklearn.metrics
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import TensorDataset, DataLoader
from collections import OrderedDict
from sklearn.model_selection import train_test_split
import  matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import numpy as np
import pandas as pd
import joblib



In [2]:
# 오토인코더 모듈 정의
class Autoencoder(nn.Module):
    def __init__(self,trdat , num_dim):
        super(Autoencoder, self).__init__()
        trdat= torch.tensor(trdat)
        data_size = trdat.size(1)
        self.num_dim = num_dim
        
        self.encoder = nn.Sequential( 
            nn.Linear(data_size, 32), 
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, self.num_dim),   
        )
        self.decoder = nn.Sequential(
            nn.Linear(self.num_dim, 8), 
            nn.ReLU(),
            nn.Linear(8, 16),
            nn.ReLU(),
            nn.Linear(16, 32),
            nn.ReLU(),
            nn.Linear(32, data_size),       
        )

    
    def fit (self, epochs, loader, criterion, optimizer):        
        
        for epoch in range(epochs):
            for step ,(x) in enumerate(loader):
                x = x[0]
                y = x
                encoded =  self.encoder(x)
                decoded =  self.decoder(encoded)
                loss = criterion(decoded , y)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            
    
    
    
    def predict(self, x):
        encoded = self.encoder(x) 
        decoded = self.decoder(encoded)  
            
        return  encoded 
                

In [3]:
def DR_AE(trdat, tsdat, epochs, num_dim):
    """
    Dimension reduction auto encoder
    Parameters
    ----------
    trdat : array
    학습 데이터

    tsdat : array
    예측 데이터
            
    epochs : int
    오토인코더 학습 횟수
            
    num_dim : int
    축소 시킬 차원수 

       

    Returns
    -------
    predarray : array
    축소된 차원의 결과값
        
    """
    
    # 학습을 위한 데이터 전처리 
    dataset = TensorDataset(torch.tensor(trdat).float())
    loader= DataLoader(dataset, batch_size=1, shuffle = True)
    
    DR_AE= Autoencoder(trdat, num_dim )
    optimizer = torch.optim.Adam(DR_AE.parameters(), lr = 0.005)
    criterion = nn.MSELoss()
    DR_AE.fit(epochs, loader, criterion, optimizer)
      
    pred_x = DR_AE.predict(torch.tensor(tsdat).float())
    predarray = pred_x.detach().numpy()
    
    # save Autoencoder model
    saved_model = joblib.dump(DR_AE, 'DR_AE.pkl')
    
    return {'predarray': predarray}

In [4]:

df = pd.read_csv('test_data.csv', encoding='euc-kr')
trdat = df.values[0:600,:]
tsdat = df.values[600:1000, :]


In [5]:
predarray = DR_AE(trdat, tsdat, 10, 2)

In [6]:
print(predarray)

{'predarray': array([[-137.23576 , -378.93375 ],
       [-137.33504 , -379.2536  ],
       [-137.30435 , -379.1545  ],
       [-137.28467 , -379.10657 ],
       [-137.28487 , -379.11465 ],
       [-137.2821  , -379.10593 ],
       [-137.38676 , -379.35263 ],
       [-137.27232 , -379.06674 ],
       [-137.20155 , -378.83887 ],
       [-142.23972 , -393.0741  ],
       [-142.1825  , -392.91992 ],
       [-139.90625 , -386.28305 ],
       [-139.93098 , -386.34494 ],
       [-137.28203 , -379.10562 ],
       [-135.36943 , -373.82993 ],
       [-137.31815 , -379.2001  ],
       [-137.2436  , -379.00723 ],
       [-137.27449 , -379.084   ],
       [-137.2687  , -379.0671  ],
       [-137.3186  , -379.1994  ],
       [-137.0493  , -378.45728 ],
       [-137.3631  , -379.3204  ],
       [-137.31874 , -379.1946  ],
       [-137.34172 , -379.25946 ],
       [-137.3007  , -379.15396 ],
       [-137.34193 , -379.27457 ],
       [-137.244   , -379.001   ],
       [-114.899475, -316.99338 ],
      